# Análise exploratória dos dados
_EDA - Exploratory Data Analysis_

---

## Sumário

1. **Importação de bibliotecas**
2. **Carregamento das bases**
3. **Divisão da base em treino, validação e teste**
    - 3.1. Salvando os dataframes em formato parquet


<br>

---

<br>

## 1. Importação de bibliotecas

In [10]:
# Importação de pacotes e definição de parâmetros globais

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import gc
import os

from sklearn.model_selection import train_test_split

In [11]:
# Configurações para exibição de dados no Jupyter Notebook

# Configurar para exibir o conteúdo completo das colunas
pd.set_option('display.max_colwidth', None)

# Configurar a supressão de mensagens de aviso durante a execução
warnings.filterwarnings('ignore')

# Configurar estilo dos gráficos do seaborn
sns.set_style('whitegrid')

## 2. Carregamento da base

In [12]:
# Efetuando a limpeza da memória antes do carregamento dos dados
print(f'\nQuantidade de objetos removidos da memória: {gc.collect()}')


Quantidade de objetos removidos da memória: 0


In [13]:
# Caminho do arquivo de dados
caminho = f'dados/ABT'

# Criando um dataframe a partir dos arquivos no diretório de dados
df = pd.read_parquet(caminho, engine='pyarrow')

In [14]:
df.head(10)

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,...,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards,code,description,transaction_id,is_fraud
0,7475806,2010-01-01 09:05:00,1840,4568,$2.02,Swipe Transaction,35451,Beaverton,OR,97005.0,...,-122.80,$21702,$44249,$103229,706,5,5812,Eating Places and Restaurants,7475806.0,No
1,7477473,2010-01-01 15:08:00,538,4161,$7.48,Swipe Transaction,26810,Winterville,NC,28590.0,...,-77.15,$14844,$30265,$36789,814,4,5541,Service Stations,7477473.0,No
2,7477784,2010-01-01 16:18:00,724,2876,$1.70,Swipe Transaction,59935,Cushing,OK,74023.0,...,-96.76,$17237,$35142,$107898,731,4,5499,Miscellaneous Food Stores,NaN,None
3,7477811,2010-01-01 16:25:00,377,1175,$-53.00,Swipe Transaction,43293,Withee,WI,54498.0,...,-122.26,$24884,$39110,$363,750,5,5499,Miscellaneous Food Stores,NaN,None
4,7478410,2010-01-01 18:55:00,1362,2145,$-295.00,Swipe Transaction,96185,Bladensburg,MD,20710.0,...,-77.27,$35563,$72510,$44317,727,4,7011,"Lodging - Hotels, Motels, Resorts",NaN,None
5,7478830,2010-01-01 21:25:00,1466,5884,$17.59,Online Transaction,16798,ONLINE,None,NaN,...,-75.61,$17624,$35933,$23451,812,2,4121,Taxicabs and Limousines,NaN,None
6,7479105,2010-01-01 23:02:00,1693,5940,$4.33,Online Transaction,85247,ONLINE,None,NaN,...,-111.89,$36300,$74016,$85204,702,2,5815,"Digital Goods - Media, Books, Apps",7479105.0,No
7,7480284,2010-01-02 11:11:00,1674,2873,$27.78,Swipe Transaction,60569,Jonesboro,AR,72401.0,...,-90.35,$14172,$26858,$11245,712,2,5300,Wholesale Clubs,7480284.0,No
8,7480339,2010-01-02 11:27:00,1070,4138,$35.20,Swipe Transaction,99256,Marion,IA,52302.0,...,-91.58,$25275,$51528,$58509,745,6,5411,"Grocery Stores, Supermarkets",NaN,None
9,7480412,2010-01-02 11:46:00,509,4588,$5.54,Swipe Transaction,60569,Charmco,WV,25958.0,...,-82.43,$21842,$44534,$107410,702,4,5300,Wholesale Clubs,7480412.0,No


## 3. Divisão da base em treino, validação e teste

In [15]:
# Criando os dataframes com base no target 'is_fraud'

# DataFrame de treino e validação (valores 'Yes' e 'No')
df_train_val = df[df['is_fraud'].isin(['Yes', 'No'])].copy()

# DataFrame de teste (valores None)
df_test = df[df['is_fraud'].isnull()].copy()

In [16]:
# Dividindo o dataframe de treino e validação, mantendo a proporção das classes do target 'is_fraud'
df_train, df_val = train_test_split(
    df_train_val, test_size=0.2, random_state=42, stratify=df_train_val['is_fraud']
)

In [17]:
# Exibindo a quantidade de linhas e colunas dos dataframes

# Criação de um dicionário com os dataframes e seus respectivos nomes
dfs = {
    'df_train': df_train,
    'df_val': df_val,
    'df_test': df_test,
}

# Iteração sobre o dicionário para exibir o nome e as dimensões dos dataframes
print(f'\nVOLUMETRIA')
for nome, df in dfs.items():
    print(f'\n{nome}')
    print(f'-'*45)
    print(f'Quantidade de linhas (registros):  {df.shape[0]}')
    print(f'Quantidade de colunas (variáveis): {df.shape[1]}')  


VOLUMETRIA

df_train
---------------------------------------------
Quantidade de linhas (registros):  3566068
Quantidade de colunas (variáveis): 43

df_val
---------------------------------------------
Quantidade de linhas (registros):  891518
Quantidade de colunas (variáveis): 43

df_test
---------------------------------------------
Quantidade de linhas (registros):  2194300
Quantidade de colunas (variáveis): 43


### 3.1. Salvando os dataframes em formato parquet

In [18]:
# Iteração sobre o dicionário para exportação dos dataframes em formato parquet
for nome, df in dfs.items():
    
    # Diretório onde os dados serão salvos
    caminho = f'dados/{nome}'
    
    # Cria o diretório se não existir
    os.makedirs(caminho, exist_ok=True)
    
    try:      
        # Exportar para Parquet
        df.to_parquet(caminho, engine='pyarrow', index=False)
        
        # Valida a quantidade de linhas e colunas
        print(f'\nO {nome} possui {df.shape[0]} linhas e {df.shape[1]} colunas')
    except Exception as e:
        print(f'\nErro ao exportar {nome}:\n {e}')


Erro ao exportar df_train:
 [WinError 5] Failed to open local file 'dados/df_train'. Detail: [Windows error 5] Acesso negado.


Erro ao exportar df_val:
 [WinError 5] Failed to open local file 'dados/df_val'. Detail: [Windows error 5] Acesso negado.


Erro ao exportar df_test:
 [WinError 5] Failed to open local file 'dados/df_test'. Detail: [Windows error 5] Acesso negado.

